In [ ]:
! pip install simpletransformers
#! pip install --upgrade traitlets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
data_dir = "gdrive/My Drive/Colab Notebooks/FakeNews/"

train_stances = pd.read_csv(data_dir+'train_stances.csv')
train_bodies = pd.read_csv(data_dir+'train_bodies.csv')

In [ ]:
train = pd.merge(train_stances, train_bodies, how='left', on='Body ID')

target = ['agree', 'disagree', 'discuss', 'unrelated']
target_dict = dict(zip(target, range(len(target))))
f = lambda x: target_dict[x]
train['label'] = [f(train['Stance'][i]) for i in range(len(train['Stance']))]

train['Headline']= train['Headline'].apply(lambda x: x.lower())
train['articleBody']= train['articleBody'].apply(lambda x: x.lower())
train.head()


# Loading test data: competition_test data
test_stances = pd.read_csv(data_dir+'competition_test_stances.csv')
test_bodies = pd.read_csv(data_dir+'competition_test_bodies.csv')
test  = pd.merge(test_stances, test_bodies, how='left', on='Body ID')
test['label'] = [f(test['Stance'][i]) for i in range(len(test['Stance']))]


test['Headline']= test['Headline'].apply(lambda x: x.lower())
test['articleBody']= test['articleBody'].apply(lambda x: x.lower())
test.head()

X_test = pd.DataFrame(test[['Headline', 'articleBody']])
y_test = test['label']

In [ ]:
train.head()

,Headline,Body ID,Stance,articleBody,label
0,police find mass graves with at least '15 bodi...,712,unrelated,danny boyle is directing the untitled film\n\n...,3
1,hundreds of palestinians flee floods in gaza a...,158,agree,hundreds of palestinians were evacuated from t...,0
2,"christian bale passes on role of steve jobs, a...",137,unrelated,30-year-old moscow resident was hospitalized w...,3
3,hbo and apple in talks for $15/month apple tv ...,1034,unrelated,(reuters) - a canadian soldier was shot at the...,3
4,spider burrowed through tourist's stomach and ...,1923,disagree,"fear not arachnophobes, the story of bunbury's...",1


In [ ]:
df_train = pd.DataFrame(columns=['text_a', 'text_b', 'labels'])
df_train['text_a'] = train['Headline']
df_train['text_b'] = train['articleBody']
df_train['labels'] = train['label']

In [ ]:
df_train.head()

,text_a,text_b,labels
0,police find mass graves with at least '15 bodi...,danny boyle is directing the untitled film\n\n...,3
1,hundreds of palestinians flee floods in gaza a...,hundreds of palestinians were evacuated from t...,0
2,"christian bale passes on role of steve jobs, a...",30-year-old moscow resident was hospitalized w...,3
3,hbo and apple in talks for $15/month apple tv ...,(reuters) - a canadian soldier was shot at the...,3
4,spider burrowed through tourist's stomach and ...,"fear not arachnophobes, the story of bunbury's...",1


In [ ]:
df_test = pd.DataFrame(columns=['headline', 'body'])
df_test['headline'] = test['Headline']
df_test['body'] = test['articleBody']

In [ ]:
df_test.head()

,headline,body
0,ferguson riots: pregnant woman loses eye after...,a respected senior french police officer inves...
1,crazy conservatives are sure a gitmo detainee ...,dave morin's social networking company path is...
2,a russian guy says his justin bieber ringtone ...,a bereaved afghan mother took revenge on the t...
3,"zombie cat: buried kitty believed dead, meows ...",hewlett-packard is officially splitting in two...
4,argentina's president adopts boy to end werewo...,an airline passenger headed to dallas was remo...


In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
model = ClassificationModel('roberta', 'roberta-base', num_labels=4, use_cuda=False, args={
    'learning_rate':1e-3,
    'num_train_epochs': 3,
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'process_count': 10,
    'evaluate_during_training_steps': False,
    'train_batch_size': 4,
    'eval_batch_size': 4,
    'max_seq_length': 512,
    'fp16': True,
    'early_stopping': True,
})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [ ]:
model.train_model(df_train)

In [ ]:
test_lst = list()

for i in range(len(df_test)):
  test_lst.append([df_test['text_a'][i], df_test['text_b'][i]])  

In [ ]:
def acc(pred):
  cnt = 0
  for i in range(len(pred)):
    if pred[i] == stances[i]:
      cnt = cnt + 1
  print(cnt / len(pred))

In [ ]:
def createAns(opt, epoch):
  df_answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])
  df_answer['Stance'] = opt
  df_answer.replace(0, 'agree',True)
  df_answer.replace(1, 'disagree',True)
  df_answer.replace(2, 'discuss',True)
  df_answer.replace(3, 'unrelated',True)
  df_answer['Headline'] = test_raw['Headline']
  df_answer['Body ID'] = test_raw['Body ID']
  fName = 'answerEp' + str(epoch) + '.csv'
  df_answer.to_csv(fName, index=False, encoding='utf-8')
  return df_answer

In [ ]:
predictions, raw_outputs = model.predict(test_lst)
acc(predictions)

createAns(predictions, 3)

In [ ]:
df_answer = pd.DataFrame(columns=['Headline', 'Body ID', 'Stance'])

In [ ]:
df_answer['Stance'] = predictions

df_answer.replace(0, 'agree',True)
df_answer.replace(1, 'disagree',True)
df_answer.replace(2, 'discuss',True)
df_answer.replace(3, 'unrelated',True)

df_answer['Headline'] = df_test['Headline']
df_answer['Body ID'] = df_test['Body ID']


In [ ]:
df_answer.to_csv(data_dir+'answer.csv', index=False, encoding='utf-8')